In [13]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


In [1]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [2]:
#load pdf files
loader = PyPDFDirectoryLoader("data/")
data = loader.load()

In [3]:
print(data)

[Document(page_content='Achieving Growth\nGaining Lasting Trust\nAnnual Report \n2022', metadata={'source': 'data\\Mobily AR_2022_English (1).pdf', 'page': 0}), Document(page_content='Achieving Growth\nGaining Lasting \nTrustAchieving growth\nMobily demonstrated strong performance this year, \nwith improvements in every financial KPI, achieving \nthe highest revenues and net profit in the last 9 \nyears. Growth in the top and bottom lines reflected \nefficient operations and continuous investment in \ndigitalization and innovation. Mobily diversified and \ninnovated its product offerings, launching its new \ndigital e-wallet, Mobily Pay. It also grew its coverage and \nimproved network quality, expanding the 5G network, \nsigning strategic partnerships, and connecting to new \nsubmarine cables. Successful execution of the GAIN \nstrategy expanded the Company’s margins and market \nvalue, while offering greater value to all Stakeholders. \nCompany growth took place in line with the grow

In [4]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [5]:
len(text_chunks)

99

In [6]:
#get the third chunk
text_chunks[2]

Document(page_content='Etihad Etisalat (Mobily) was established in 2004 through Saudi \nArabia’s second Global System for Mobile Communications (GSM) \nlicense, ending the monopoly in the Saudi wireless industry and \nproviding choice in national mobile telecommunications services \nto the Saudi population for the first time.About Mobily\nThe Company has rapidly grown and diversified both \norganically and through strategic acquisitions over the \nyears, driven by investment in cutting-edge infrastructure – \nlaunching 3.5G services in 2006, 4G services in 2011, and 5G \nservices in 2019 – along with its commitment to continuous \ninnovation to drive growth and deliver excellence in the \ncustomer experience. \nMobily provides integrated telecommunications services \nto serve its diverse customer base across Saudi Arabia, \nincluding individuals, businesses and carriers. It delivers \nservices through its modern wireless network, which is \namong the largest by coverage in Saudi Arabia

In [7]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


c:\Users\sneha.shukla\Desktop\pdf_chatbot\MobilyAR-PDF-Chatbot\pdf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [9]:
!pip install llama-cpp-python

In [9]:
#Import Model
# from llama_cpp import Llama
from langchain.llms import LlamaCpp
llm = LlamaCpp(
    streaming=True,
    model_path="Models\mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.1,
    top_p=1,
    verbose=True,
    n_ctx=4096
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from Models\mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 lla

In [10]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [ ]:
query = "What is python"

In [ ]:
qa.run(query)

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")